In [1]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from google.cloud.dataproc_v1 import Session


# This will create a default Spark session
spark = DataprocSparkSession.builder.getOrCreate()
# If you would like to customize the Spark session, please refer to the
# documentation at https://cloud.google.com/bigquery/docs/use-spark
#
# For example, if you need to use a different subnetwork, use the code below

# from google.cloud.dataproc_v1 import Session

# session = Session()
# session.environment_config.execution_config.subnetwork_uri = "<subnetwork_name>"
# spark = DataprocSparkSession.builder.dataprocSessionConfig(session).getOrCreate()

████████████████████████████████████████████████▌                               

KeyboardInterrupt: 

In [2]:
%%bigquery

select count(1) from kaggle-gcp-comp1.APP_REVIEWS.APP_REVIEWS

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,2514


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pratyushpuri/multilingual-mobile-app-reviews-dataset-2025")

print("Path to dataset files:", path)

100%|██████████| 133k/133k [00:00<00:00, 45.6MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/pratyushpuri/multilingual-mobile-app-reviews-dataset-2025/versions/1


In [ ]:
!cp -r /root/.cache/kagglehub/datasets/pratyushpuri/multilingual-mobile-app-reviews-dataset-2025/versions/1/ /content/multilingual-mobile-app-review/

In [ ]:
!gcloud storage cp -r /content/multilingual-mobile-app-review/ gs://kaggal-gcp-comp1-data/

Copying file:///content/multilingual-mobile-app-review/multilingual_mobile_app_reviews_2025.csv to gs://kaggal-gcp-comp1-data/multilingual-mobile-app-review/multilingual_mobile_app_reviews_2025.csv


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lava18/google-play-store-apps")

print("Path to dataset files:", path)



100%|██████████| 1.94M/1.94M [00:00<00:00, 91.0MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/lava18/google-play-store-apps/versions/6


In [ ]:
!cp -r /root/.cache/kagglehub/datasets/lava18/google-play-store-apps/versions/6 /content/google-play-store-apps/

In [ ]:
!gcloud storage cp -r /content/google-play-store-apps gs://kaggal-gcp-comp1-data/google-play-store-apps

Copying file:///content/google-play-store-apps/googleplaystore_user_reviews.csv to gs://kaggal-gcp-comp1-data/google-play-store-apps/googleplaystore_user_reviews.csv
Copying file:///content/google-play-store-apps/license.txt to gs://kaggal-gcp-comp1-data/google-play-store-apps/license.txt
Copying file:///content/google-play-store-apps/googleplaystore.csv to gs://kaggal-gcp-comp1-data/google-play-store-apps/googleplaystore.csv

Average throughput: 40.6MiB/s


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ramamet4/app-store-apple-data-set-10k-apps")

print("Path to dataset files:", path)

100%|██████████| 5.63M/5.63M [00:00<00:00, 141MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/ramamet4/app-store-apple-data-set-10k-apps/versions/7


In [ ]:
!cp -r /root/.cache/kagglehub/datasets/ramamet4/app-store-apple-data-set-10k-apps/versions/7/ /content/apple-play-store-apps/

In [ ]:
!gcloud storage cp -r /content/apple-play-store-apps gs://kaggal-gcp-comp1-data/apple-play-store-apps

Copying file:///content/apple-play-store-apps/AppleStore.csv to gs://kaggal-gcp-comp1-data/apple-play-store-apps/AppleStore.csv
Copying file:///content/apple-play-store-apps/appleStore_description.csv to gs://kaggal-gcp-comp1-data/apple-play-store-apps/appleStore_description.csv

Average throughput: 69.1MiB/s


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("quadeer15sh/windows-store-top-apps-games")

print("Path to dataset files:", path)

100%|██████████| 1.85M/1.85M [00:00<00:00, 101MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/quadeer15sh/windows-store-top-apps-games/versions/1


In [ ]:
!cp -r /root/.cache/kagglehub/datasets/quadeer15sh/windows-store-top-apps-games/versions/1 /content/windows-store-apps/



In [ ]:
!gcloud storage cp -r /content/windows-store-apps gs://kaggal-gcp-comp1-data/

Copying file:///content/windows-store-apps/windows_store.csv to gs://kaggal-gcp-comp1-data/windows-store-apps/windows_store.csv


In [ ]:
from pyspark.sql.functions import sum, column as col, array_agg

In [ ]:
df = spark.read.csv('gs://kaggal-gcp-comp1-data/multilingual_mobile_app_reviews_2025.csv',
                    inferSchema=True,
                    header=True)

In [ ]:
df.columns

['review_id',
 'user_id',
 'app_name',
 'app_category',
 'review_text',
 'review_language',
 'rating',
 'review_date',
 'verified_purchase',
 'device_type',
 'num_helpful_votes',
 'user_age',
 'user_country',
 'user_gender',
 'app_version']

In [ ]:
df.schema

StructType([StructField('review_id', IntegerType(), True), StructField('user_id', IntegerType(), True), StructField('app_name', StringType(), True), StructField('app_category', StringType(), True), StructField('review_text', StringType(), True), StructField('review_language', StringType(), True), StructField('rating', DoubleType(), True), StructField('review_date', TimestampType(), True), StructField('verified_purchase', BooleanType(), True), StructField('device_type', StringType(), True), StructField('num_helpful_votes', IntegerType(), True), StructField('user_age', DoubleType(), True), StructField('user_country', StringType(), True), StructField('user_gender', StringType(), True), StructField('app_version', StringType(), True)])

In [ ]:
print(df.count()) #2514

2514


In [ ]:
df_cnt = df.groupBy("app_name").count()

In [ ]:
df_cnt.select(sum(col("count"))).show()

+----------+
|sum(count)|
+----------+
|      2514|
+----------+



In [ ]:
df.where(df.app_name=='Udemy').count()

np.int64(52)

In [ ]:
df.summary().show()

+-------+-----------------+------------------+---------------+--------------------+-------------------------------------+---------------+------------------+-----------+-----------------+------------------+------------+-----------------+------------------+
|summary|        review_id|           user_id|       app_name|        app_category|                          review_text|review_language|            rating|device_type|num_helpful_votes|          user_age|user_country|      user_gender|       app_version|
+-------+-----------------+------------------+---------------+--------------------+-------------------------------------+---------------+------------------+-----------+-----------------+------------------+------------+-----------------+------------------+
|  count|             2514|              2514|           2514|                2514|                                 2455|           2514|              2477|       2514|             2514|              2514|        2473|             1

In [ ]:
df.describe().show()

+-------+-----------------+------------------+---------------+--------------------+-------------------------------------+---------------+------------------+-----------+-----------------+------------------+------------+-----------------+------------------+
|summary|        review_id|           user_id|       app_name|        app_category|                          review_text|review_language|            rating|device_type|num_helpful_votes|          user_age|user_country|      user_gender|       app_version|
+-------+-----------------+------------------+---------------+--------------------+-------------------------------------+---------------+------------------+-----------+-----------------+------------------+------------+-----------------+------------------+
|  count|             2514|              2514|           2514|                2514|                                 2455|           2514|              2477|       2514|             2514|              2514|        2473|             1

In [ ]:
unique_app_cat_df = df.select(df.app_name,df.app_category).drop_duplicates()

In [ ]:
unique_app_cat_agg_df = unique_app_cat_df.groupby("app_name").agg(array_agg("app_category").alias("app_categories_array"))

In [ ]:
unique_app_cat_list = [{'app_name':row.app_name, 'app_category':row.app_categories_array} for row in unique_app_cat_agg_df.collect()]

In [ ]:
from google.genai import client, models

genai_client = client.Client(vertexai=True,project="kaggle-gcp-comp1", location='us-central1')

In [ ]:
def get_category(app_name:str, categories:list)->str:
  categories_str = ",".join(categories)
  system_instructions = """I give you a android app name and a list of app categories.you have correctly choose which category the app belongs.
  Just return only the correct category.
  app_name : {app_name}
  app_categoryies : {categories}
  """
  prompt = system_instructions.format(app_name=app_name, categories = categories_str)

  response = genai_client.models.generate_content(model="gemini-2.5-flash-lite", contents=prompt)
  return response.text

In [ ]:
unique_app_cat_list_genai = []
for item in unique_app_cat_list:
  correct_category = get_category(item['app_name'], item['app_category'])
  unique_app_cat_list_genai.append({'app_name':item['app_name'], 'app_category':correct_category})

In [ ]:
unique_app_cat_list_genai

[{'app_name': 'Udemy', 'app_category': 'Education'},
 {'app_name': 'Google Maps', 'app_category': 'Travel & Local'},
 {'app_name': 'Grammarly', 'app_category': 'Productivity'},
 {'app_name': 'Khan Academy', 'app_category': 'Education'},
 {'app_name': 'Pinterest', 'app_category': 'Social Networking'},
 {'app_name': 'Airbnb', 'app_category': 'Travel & Local'},
 {'app_name': 'Bumble', 'app_category': 'Dating'},
 {'app_name': 'Reddit', 'app_category': 'Social Networking'},
 {'app_name': 'Lyft', 'app_category': 'Travel & Local'},
 {'app_name': 'Google Drive', 'app_category': 'Productivity'},
 {'app_name': 'Adobe Photoshop', 'app_category': 'Photography'},
 {'app_name': 'TikTok', 'app_category': 'Social Networking'},
 {'app_name': 'Coursera', 'app_category': 'Education'},
 {'app_name': 'MX Player', 'app_category': 'Video Players & Editors'},
 {'app_name': 'Twitter', 'app_category': 'Social Networking'},
 {'app_name': 'WhatsApp', 'app_category': 'Communication'},
 {'app_name': 'Booking.com', 

In [ ]:
correct_categories = spark.createDataFrame(unique_app_cat_list_genai)

In [ ]:
correct_categories.count()

np.int64(41)

In [ ]:
spark.conf.set("temporaryGcsBucket","kaggal-gcp-comp1-data")

In [ ]:
correct_categories.write.mode("overwrite").format('bigquery').save('kaggle-gcp-comp1.APP_REVIEWS.correct_app_categories')

In [ ]:
correct_categories = correct_categories.withColumnsRenamed({"app_category":"app_category_new","app_name":"app_name_new"})

In [ ]:
joined_df = df.join(correct_categories,
                    df.app_name == correct_categories.app_name_new,
                    how='left')

In [ ]:
joined_df = joined_df.withColumn("app_category", joined_df.app_category_new).drop("app_category_new","app_name_new")

In [ ]:

joined_df.write.mode("overwrite").format('bigquery').save('kaggle-gcp-comp1.APP_REVIEWS.APP_REVIEWS')